In [2]:
# !pip install MetaTrader5

In [45]:
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import pandas as pd
import time
from tqdm.notebook import tqdm

In [8]:
if not mt5.initialize():
    mt5.shutdown()
mt5.terminal_info()

TerminalInfo(community_account=False, community_connection=False, connected=True, dlls_allowed=False, trade_allowed=True, tradeapi_disabled=False, email_enabled=False, ftp_enabled=False, notifications_enabled=False, mqid=False, build=2875, maxbars=100000, codepage=0, ping_last=284198, community_balance=0.0, retransmission=0.0, company='FX Choice Limited', name='FXChoice MetaTrader 5', language='English', path='C:\\Program Files\\FXChoice MetaTrader 5', data_path='C:\\Users\\samch\\AppData\\Roaming\\MetaQuotes\\Terminal\\9EBAECC50A1974DB0763442DEE6D2442', commondata_path='C:\\Users\\samch\\AppData\\Roaming\\MetaQuotes\\Terminal\\Common')

In [75]:
data = [_.split('\t') for _ in """

	 ### paste here ####



EURNZD	1.68373	0.08126298797	0.001214369557
EURNZD	1.66144	0.08126298797	0.001214369557
GBPAUD	1.82138	0.05071160639	0.001811063391
		#N/A	#N/A
		#N/A	#N/A
GBPAUD	1.80414	0.05071160639	0.001811063391
		#N/A	#N/A
NZDUSD	0.72639	0.01999634707	0.00356065034
		#N/A	#N/A
		#N/A	#N/A
NZDUSD	0.7148	0.01999634707	0.00356065034
EURUSD	1.21582	0.09323861443	0.0007636321114
USDCHF	0.90004	0.06385377114	0.001005920414




""".split('\n') if _.split('\t')[0] != '']

data = [
    [
        _[0]
        , float(_[1])
        , round(float(_[2]), 2)
        , round(float(_[3]), 6)
    ] for _ in data
]

data

[['EURNZD', 1.68373, 0.08, 0.001214],
 ['EURNZD', 1.66144, 0.08, 0.001214],
 ['GBPAUD', 1.82138, 0.05, 0.001811],
 ['GBPAUD', 1.80414, 0.05, 0.001811],
 ['NZDUSD', 0.72639, 0.02, 0.003561],
 ['NZDUSD', 0.7148, 0.02, 0.003561],
 ['EURUSD', 1.21582, 0.09, 0.000764],
 ['USDCHF', 0.90004, 0.06, 0.001006]]

In [76]:
# for inst in data (unique),
# get current price
# if pending above, sell
# if pending below, buy

In [77]:
p = []
for i in tqdm(set([_[0] for _ in data])):
    a = mt5.symbol_info_tick(i)
    p.append([i, a])

p = [
    [
        _[0], 
        round((_[1].bid + _[1].ask) / 2, 6)
    ] for _ in p
]

p

[['EURUSD', 1.213265],
 ['USDCHF', 0.902155],
 ['NZDUSD', 0.721585],
 ['EURNZD', 1.68135],
 ['GBPAUD', 1.817345]]

In [78]:
data

[['EURNZD', 1.68373, 0.08, 0.001214],
 ['EURNZD', 1.66144, 0.08, 0.001214],
 ['GBPAUD', 1.82138, 0.05, 0.001811],
 ['GBPAUD', 1.80414, 0.05, 0.001811],
 ['NZDUSD', 0.72639, 0.02, 0.003561],
 ['NZDUSD', 0.7148, 0.02, 0.003561],
 ['EURUSD', 1.21582, 0.09, 0.000764],
 ['USDCHF', 0.90004, 0.06, 0.001006]]

In [79]:
results = []
for _ in data:
    if _[1] > [x[1] for x in p if x[0] == _[0]][0]:
        print('sell', _, )
        
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_SELL_LIMIT,
            "price": _[1],
            "sl": _[1] + _[3],
            "tp": _[1] - (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    else:
        print('buy', _, )
        r = {
            "action": mt5.TRADE_ACTION_PENDING,
            "symbol": _[0],
            "volume": _[2],
            "type": mt5.ORDER_TYPE_BUY_LIMIT,
            "price": _[1],
            "sl": _[1] - _[3],  ############ this is getting mixed up for shorts
            "tp": _[1] + (_[3]*3),
            "deviation": 10,
            "magic": 9,
            "comment": "sent by python",
            "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
            "type_filling": mt5.ORDER_FILLING_RETURN,
        }
        result = mt5.order_send(r)  
    results.append(result)

sell ['EURNZD', 1.68373, 0.08, 0.001214]
buy ['EURNZD', 1.66144, 0.08, 0.001214]
sell ['GBPAUD', 1.82138, 0.05, 0.001811]
buy ['GBPAUD', 1.80414, 0.05, 0.001811]
sell ['NZDUSD', 0.72639, 0.02, 0.003561]
buy ['NZDUSD', 0.7148, 0.02, 0.003561]
sell ['EURUSD', 1.21582, 0.09, 0.000764]
buy ['USDCHF', 0.90004, 0.06, 0.001006]


In [80]:
results

[OrderSendResult(retcode=10009, deal=0, order=5635940, volume=0.08, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=2, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURNZD', volume=0.08, price=1.68373, stoplimit=0.0, sl=1.684944, tp=1.680088, deviation=10, type=3, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0)),
 OrderSendResult(retcode=10009, deal=0, order=5635941, volume=0.08, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=3, retcode_external=0, request=TradeRequest(action=5, magic=9, order=0, symbol='EURNZD', volume=0.08, price=1.66144, stoplimit=0.0, sl=1.660226, tp=1.665082, deviation=10, type=2, type_filling=2, type_time=0, expiration=0, comment='sent by python', position=0, position_by=0)),
 OrderSendResult(retcode=10009, deal=0, order=5635942, volume=0.05, price=0.0, bid=0.0, ask=0.0, comment='Request executed', request_id=4, retcode_external=0, request=Trade